**IMPORTING LIBRARIES**

In [ ]:
import pandas as pd
import numpy as np

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T
import PIL

from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

**DATA TRANSFORM**

In [ ]:
transform = T.Compose(
                [T.ToTensor(),T.Normalize((0.5,),(0.5,))]
                     )

**LOADING CIFAR10 DATASET**

In [ ]:
import torchvision.datasets as dset
from torch.utils.data import sampler,DataLoader

class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset. 
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start = 0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples


NUM_TRAIN = 49000
NUM_VAL = 1000

trainSet = dset.CIFAR10(root = 'OneDrive/Documents/Machine Learning/DL/', 
                             train=True, 
                             download=True,
                             transform=transform)

trainLoader = DataLoader(trainSet, 
                          batch_size=128, 
                          sampler=ChunkSampler(NUM_TRAIN, 0))

valSet = dset.CIFAR10(root = 'OneDrive/Documents/Machine Learning/DL/', 
                           train=True, 
                           download=True,
                           transform=transform)

valLoader = DataLoader(valSet, 
                        batch_size=128, 
                        sampler=ChunkSampler(NUM_VAL, NUM_TRAIN))

testSet = dset.CIFAR10(root = 'OneDrive/Documents/Machine Learning/DL/', 
                            train=False, 
                            download=True,
                            transform=transform)

testLoader = DataLoader(testSet, 
                         batch_size=128)
classes = ('airplane', 'automobile', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

**VISUALIZING DATA**

In [ ]:
fig = plt.figure(figsize = (12,12))
for i in range(1,17):
    plt.subplot(4,4,i)
    plt.imshow(trainSet.data[i])

In [ ]:
print(trainSet.data.shape)
print(testSet.data.shape)

**TRAINING and ACCURACY FUNCTIONS**

In [ ]:
def train(net,loss_func,opt,lr,num_epoch=1):
    iteration = 0
    loss_list = []
    iter_list = []
    val_acc_list = []
    for epoch in range(num_epoch):
        print("Epoch %d/%d"%(epoch+1,num_epoch))
        net.train()
        for i,data in enumerate(trainLoader,0):
            torch.cuda.empty_cache()
            inputs, labels = data

            # Using GPU
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Wrapping in Variable
            inputs, labels = Variable(inputs), Variable(labels)

            # Clearing previous gradients
            opt.zero_grad()

            # Forward Propagation
            scores = net(inputs)
            loss = loss_func(scores,labels)

            # Backpropagation
            loss.backward()
            opt.step()

            iteration += 1
            if iteration % 100 == 0:
                loss_list.append(loss)
                iter_list.append(iteration)
                # Print Loss
                print('Iteration: %d, loss = %0.7f'%(iteration,loss))
            
        val_acc = check_accuracy(net,valLoader)
        val_acc_list.append(val_acc)
        if (epoch+1) % 15 == 0:
            lr /= 3
            update_lr(opt, lr)
    
    return iter_list,loss_list,val_acc_list

In [ ]:
def vizualize(iter_l,loss_l,val_l,epochs=1):
    # visualizing loss function
    plt.plot(iter_l,loss_l)
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.title('Optimizing Lost function over CIFAR10 Dataset')
    plt.show()

    #visualizing Val Accuracy
    plt.plot(np.arange(epochs),val_l)
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy over Validation Set')
    plt.show()

In [ ]:
def check_accuracy(model,loader):
    if loader.dataset.train:
        print('Checking accuracy on Validation set')
    else:
        print('Checking accuracy on Test set')
    num_correct = 0
    num_samples = 0
    model.eval()
    for i,data in enumerate(loader,0):
            torch.cuda.empty_cache()
            inputs, labels = data
            
            # Using GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Forward Propagation
            scores = model(inputs)
            # Get predictions from the maximum value
            predicted = torch.max(scores.data, 1)[1]
                
            # Total number of labels
            num_samples += len(labels)
                
            num_correct += (predicted == labels).sum()
            
    accuracy = 100 * num_correct / float(num_samples)
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, accuracy))
    return accuracy

**CNN MODEL FOR CIFAR10**

In [ ]:
def conv3x3(in_channels,out_channels,stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

class ResidualBlock(nn.Module):
    def __init__(self,in_channels,out_channels,stride=1,downsample=None):
        super(ResidualBlock,self).__init__()
        self.conv1 = conv3x3(in_channels,out_channels,stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels,out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self,x):
        res = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            res = self.downsample(x)
        out += res
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    
    def __init__(self,block,layers,num_classes=10):
        super(ResNet,self).__init__()
        self.in_channels = 16
        
        self.conv = conv3x3(3,16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block,16,layers[0])
        self.layer2 = self.make_layer(block,32,layers[1],2)
        self.layer3 = self.make_layer(block,64,layers[2],2)
        self.layer4 = self.make_layer(block,128,layers[3],2)
        self.maxpool = nn.MaxPool2d(4)
        self.fc = nn.Linear(128,num_classes)
        
    
    def make_layer(self,block,out_channels,layer,stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                            conv3x3(self.in_channels,out_channels,stride=stride),
                            nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels,out_channels,stride,downsample))
        self.in_channels = out_channels
        for i in range(1,layer):
            layers.append(block(out_channels,out_channels))
        return nn.Sequential(*layers)
        
        
    def forward(self,x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.maxpool(out)
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        return out

In [ ]:
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

**GPU!**

In [ ]:
print("GPU Available:",torch.cuda.is_available())
print("Number of GPU available:",torch.cuda.device_count())

In [ ]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    print("Running on the CPU")

**DEFINING NET, LOSS AND OPTIMIZER**

In [ ]:
num_epoch = 45
learning_rate = 1e-3

In [ ]:
net = ResNet(ResidualBlock,[2,2,2,2]).to(device)
loss_func = nn.CrossEntropyLoss()
opt = optim.Adam(params = net.parameters(), lr=learning_rate)

**TRAINING and VIZUALISATION of MODEL**

In [ ]:
a,b,c = train(net,loss_func,opt,learning_rate,num_epoch)

In [ ]:
vizualize(a,b,c,num_epoch)

In [ ]:
_ = check_accuracy(net,testLoader)

In [ ]:
check_accuracy(net,trainLoader)

**SAVING MODEL**

In [ ]:
torch.save(net, 'resnet_cifar10.pth')